# Preparations

Import various libraries: Mainly Tensorflow & Keras

> **TODO: Remove unnecessary imports**

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential,Input,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Concatenate,GlobalAveragePooling2D,Lambda,ZeroPadding2D

Using TensorFlow backend.


Setting some general values.

In [3]:
IMG_HEIGHT = 200 # original is 300
IMG_WIDTH = 200

BATCH_SIZE_TRAIN = 2
BATCH_SIZE_VAL = 2
EPOCHS = 2

# Load the Images

First we load the classified images.

In [4]:
dataset_dir = './dataset/'

img_gen = ImageDataGenerator(
    rescale = 1./255, # Rescale [0, 255] to [0, 1] because NN prefer smaller input
    horizontal_flip = True, # Randomly flip inputs horizontally
    vertical_flip = True, # Randomly flips inputs vertically
    height_shift_range = .2,
    validation_split = 0.2 # mark  20% as belonging to the validation set
)

train_gen = img_gen.flow_from_directory(
    dataset_dir,
    batch_size=BATCH_SIZE_TRAIN,
    shuffle = True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical',
    subset = 'training'
)

val_gen = img_gen.flow_from_directory(
    dataset_dir,
    batch_size=BATCH_SIZE_VAL,
    shuffle = True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode = 'categorical',
    subset = 'validation'
)

print(train_gen[0])


Found 1751 images belonging to 3 classes.
Found 437 images belonging to 3 classes.
(array([[[[0.10466022, 0.4817419 , 0.17585953],
         [0.09803922, 0.4784314 , 0.17254902],
         [0.09803922, 0.48235297, 0.18431373],
         ...,
         [0.6156863 , 0.4666667 , 0.32156864],
         [0.6223073 , 0.4732877 , 0.32818964],
         [0.63015044, 0.48444134, 0.33934328]],

        [[0.10919286, 0.48566347, 0.1797811 ],
         [0.09472872, 0.4751209 , 0.16923854],
         [0.09472872, 0.47904247, 0.18100324],
         ...,
         [0.6322388 , 0.48652968, 0.34143165],
         [0.63015044, 0.48444134, 0.33934328],
         [0.6413041 , 0.49289554, 0.34779748]],

        [[0.10318293, 0.48958504, 0.18701316],
         [0.09742815, 0.48113084, 0.18186946],
         [0.09742815, 0.4817419 , 0.18370266],
         ...,
         [0.63529414, 0.4901961 , 0.33847708],
         [0.6413041 , 0.49289554, 0.34117648],
         [0.6497583 , 0.49742818, 0.33908814]],

        ...,

        

We have 2188 images in total.
They have now been split up in training and validation sets (that are strictly not overlapping)

# Create the model

This is the part that has to be optimized the most! (and isn't at all yet)

In [21]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='sigmoid')
])

# Compile the model

In [22]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 200, 200, 16)      448       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 100, 100, 16)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 100, 100, 32)      4640      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 50, 50, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 50, 50, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 25, 25, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 40000)            

In [23]:
history = model.fit_generator(
    train_gen,
    steps_per_epoch=BATCH_SIZE_TRAIN,
    epochs=EPOCHS,
    validation_data=val_gen,
    validation_steps=BATCH_SIZE_VAL
)

Epoch 1/2
2/2 [==============================] - 1s 444ms/step - loss: 0.5498 - accuracy: 0.6667 - val_loss: 2.0117 - val_accuracy: 0.6667
Epoch 2/2
2/2 [==============================] - 1s 363ms/step - loss: 2.5336 - accuracy: 0.3333 - val_loss: 0.7895 - val_accuracy: 0.6667


Evaluating the result

In [24]:
scores = model.evaluate_generator(val_gen, steps=BATCH_SIZE_VAL)
print("Model Test Loss:" , scores[0])
print("Model Test Accuracy:", scores[1])

Model Test Loss: 0.9187431335449219
Model Test Accuracy: 0.6666666865348816


# Saving the model

We're using the SavedModel format as that also saves all of the weights 

In [27]:
model_name = "akt-model"

#model.save(model_name + ".h5")

predictions = model.predict(train_gen)

print(predictions)
# test

# can be loaded again with
# from keras.models import load_model
# model = load_model(model_name + ".h5")

[[0.11827085 0.1647515  0.0332455 ]
 [0.12331837 0.13666241 0.02759341]
 [0.11325223 0.17751849 0.03311683]
 ...
 [0.09788693 0.13223208 0.01967652]
 [0.11063512 0.15946276 0.01858083]
 [0.10527228 0.14743868 0.02171037]]
